In [1]:
import osmnx as ox
import folium
from shapely.geometry import shape
import sys

sys.path.append("../src/")
from utils import (
    get_route,
    get_nearest_station,
    get_valencian_open_data,
    get_valenbisi_stations,
)

cycling_graph = ox.load_graphml("../data/valencia_cycling_network.graphml")
walking_graph = ox.load_graphml("../data/valencia_walking_network.graphml")


def get_valenbisi_route(start, end, cycling_graph, walking_graph, valenbisi_stations):
    """
    Get the Valenbisi route from start to end using the cycling network graph.

    Parameters:
        start(tuple): Tuple of coordinates for the start point.
        end(tuple): Tuple of coordinates for the end point.
        cycling_graph: The cycling network graph.
        walking_graph: The walking network graph.
        valenbisi_stations(geoDataFrame): A GeoDataFrame containing Valenbisi stations.

    Returns:
        tuple: A tuple containing three lists:
            - The walking route from the start to the nearest Valenbisi station.
            - The cycling route between the two Valenbisi stations.
            - The walking route from the nearest Valenbisi station to the end point.
            - The nearest Valenbisi station to the start point.
            - The nearest Valenbisi station to the end point.
    """

    threshold = 0.0001

    cycling_nearest_node = ox.distance.nearest_nodes(
        cycling_graph, X=start[1], Y=start[0]
    )

    ini_valenbisi_station = valenbisi_stations[
        valenbisi_stations["available"] > 0
    ].copy()
    end_valenbisi_station = valenbisi_stations[valenbisi_stations["free"] > 0].copy()

    ini_station = get_nearest_station(
        (
            cycling_graph.nodes[cycling_nearest_node]["y"],
            cycling_graph.nodes[cycling_nearest_node]["x"],
        ),
        ini_valenbisi_station,
    )
    end_station = get_nearest_station(end, end_valenbisi_station)

    ini_station_loc = ini_station["geo_point_2d"]
    end_station_loc = end_station["geo_point_2d"]

    ini_walking_route = get_route(
        start, (ini_station_loc["lat"], ini_station_loc["lon"]), walking_graph
    )
    end_walking_route = get_route(
        (end_station_loc["lat"], end_station_loc["lon"]), end, walking_graph
    )
    cycling_route = get_route(
        (ini_station_loc["lat"], ini_station_loc["lon"]),
        (end_station_loc["lat"], end_station_loc["lon"]),
        cycling_graph,
    )

    dist_ini_station = ox.distance.euclidean(
        ini_station_loc["lat"],
        ini_station_loc["lon"],
        cycling_graph.nodes[cycling_route[0]]["y"],
        cycling_graph.nodes[cycling_route[0]]["x"],
    )

    dist_end_station = ox.distance.euclidean(
        end_station_loc["lat"],
        end_station_loc["lon"],
        cycling_graph.nodes[cycling_route[-1]]["y"],
        cycling_graph.nodes[cycling_route[-1]]["x"],
    )

    if dist_ini_station > threshold:
        inter_ini = get_route(
            (ini_station_loc["lat"], ini_station_loc["lon"]),
            (
                cycling_graph.nodes[cycling_route[0]]["y"],
                cycling_graph.nodes[cycling_route[0]]["x"],
            ),
            walking_graph,
        )
        ini_walking_route.extend(inter_ini)
    if dist_end_station > threshold:
        inter_end = get_route(
            (
                cycling_graph.nodes[cycling_route[-1]]["y"],
                cycling_graph.nodes[cycling_route[-1]]["x"],
            ),
            (end_station_loc["lat"], end_station_loc["lon"]),
            walking_graph,
        )
        end_walking_route = inter_end + end_walking_route

    return ini_walking_route, cycling_route, end_walking_route, ini_station, end_station

In [2]:
def print_route(route, graph, map, color="blue"):
    """
    Print the route on the map.

    Parameters:
        route (list): List of nodes in the route.
        graph: The graph containing the nodes.
        map: The folium map to draw the route on.
        color (str): Color of the route line on the map.
    """
    if not route:
        print("No route found.")
        return

    for i in range(len(route) - 1):
        edge_data = graph.get_edge_data(route[i], route[i + 1])
        if edge_data and "geometry" in edge_data[0]:
            geom = edge_data[0]["geometry"]
            folium.PolyLine(
                locations=[(point[1], point[0]) for point in geom.coords],
                color=color,
                weight=5,
            ).add_to(map)
        else:
            folium.PolyLine(
                locations=[
                    (
                        graph.nodes[route[i]]["y"],
                        graph.nodes[route[i]]["x"],
                    ),
                    (
                        graph.nodes[route[i + 1]]["y"],
                        graph.nodes[route[i + 1]]["x"],
                    ),
                ],
                color=color,
                weight=5,
            ).add_to(map)

In [3]:
params = {
    "rows": 100,
}
url = "https://valencia.opendatasoft.com//api/explore/v2.1/catalog/datasets/valenbisi-disponibilitat-valenbisi-dsiponibilidad/records"
valenbisi_stations = get_valencian_open_data(url, params)

valenbisi_stations["geometry"] = valenbisi_stations["geo_shape"].apply(shape)
valenbisi_stations = get_valenbisi_stations(valenbisi_stations)
valenbisi_stations = valenbisi_stations[valenbisi_stations["open"] == "T"]

start = (39.4699, -0.3763)  # Example start coordinates (Valencia city center)
end = (39.48098, -0.3484897)  # Example end coordinates (nearby location)


ini_walking_route, cycling_route, end_walking_route, ini_station, end_station = (
    get_valenbisi_route(start, end, cycling_graph, walking_graph, valenbisi_stations)
)

m = folium.Map(location=start, zoom_start=14)


# Dibujar las rutas en el mapa
print_route(ini_walking_route, walking_graph, m, color="green")
print_route(cycling_route, cycling_graph, m, color="blue")
print_route(end_walking_route, walking_graph, m, color="green")


ini_station_loc = ini_station["geo_point_2d"]
end_station_loc = end_station["geo_point_2d"]

# Salida y llegada
folium.Marker(
    location=(start[0], start[1]),
    popup="Start",
    icon=folium.Icon(color="blue"),
).add_to(m)

folium.Marker(
    location=(end[0], end[1]),
    popup="End",
    icon=folium.Icon(color="blue"),
).add_to(m)


# Cambiar el marquer de forma que muestre una bici y un número de bicis en el propio marcador
folium.Marker(
    location=(ini_station_loc["lat"], ini_station_loc["lon"]),
    popup=f"Start Station.<br>Available Bikes: {ini_station['available']}",
    icon=folium.Icon(color="green", icon="bicycle", prefix="fa"),
).add_to(m)


folium.Marker(
    location=(end_station_loc["lat"], end_station_loc["lon"]),
    popup=f"End Station: {end_station['address']}<br>Available Places: {end_station['free']}",
    icon=folium.Icon(color="red", icon="home"),
).add_to(m)

m

In [4]:
edge_data = cycling_graph.get_edge_data(cycling_route[0], cycling_route[1])

if edge_data and "geometry" in edge_data[0]:
    geom = edge_data[0]["geometry"]
    for point in geom.coords:
        print(point)

(-0.3767478, 39.4673796)
(-0.3766959, 39.4673769)
(-0.3766565, 39.4673747)
